# Dash practice with recipes from the Allrecipes project

Follow-on work from the tutorial from realpython.com\
https://www.justintodata.com/python-interactive-dashboard-with-plotly-dash-tutorial/

Ideas / Plan:
- [x] Make sugar column into number
- [x] Plot average review vs amount of sugar
- [x] Interactivity: can choose y (avg_rating or no_ratings) and x (sugar content or calories)

# Part 0: prep data

### Import libraries

In [1]:
import pandas as pd
import numpy as np

In [2]:
# read in data
data = pd.read_csv("processed_recipe_data.csv")

In [3]:
data.head(1)

,recipe_id,title,date_published,description,avg_rating,ratings_no,recipe_cats,5 stars,4 stars,3 stars,...,nutrition.cholesterolContent,nutrition.fatContent,nutrition.fiberContent,nutrition.proteinContent,nutrition.saturatedFatContent,nutrition.servingSize,nutrition.sodiumContent,nutrition.sugarContent,nutrition.transFatContent,nutrition.unsaturatedFatContent
0,10004,Pavlova,2019-04-04,My grandmother's recipe from New Zealand for t...,4.52,25,"['Dessert Recipes', 'Specialty Dessert Recipes...",18,4,1,...,NaN,NaN,NaN,1.4 g,NaN,NaN,21.3 mg,25.2 g,NaN,NaN


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3391 entries, 0 to 3390
Data columns (total 35 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   recipe_id                        3391 non-null   int64  
 1   title                            3391 non-null   object 
 2   date_published                   3391 non-null   object 
 3   description                      3391 non-null   object 
 4   avg_rating                       3391 non-null   float64
 5   ratings_no                       3391 non-null   int64  
 6   recipe_cats                      3391 non-null   object 
 7   5 stars                          3391 non-null   int64  
 8   4 stars                          3391 non-null   int64  
 9   3 stars                          3391 non-null   int64  
 10  2 stars                          3391 non-null   int64  
 11  1 star                           3391 non-null   int64  
 12  reviews_no          

In [5]:
# turn relevant numeric columns into numbers
data['nutrition.sugarContent.g'] = data['nutrition.sugarContent'].str.replace(r' g', '').astype(float)
data['nutrition.calories.kcal'] = data['nutrition.calories'].str.replace(r' calories', '').astype(float)

In [6]:
# check
data[['nutrition.sugarContent', 'nutrition.sugarContent.g']].head(3)

,nutrition.sugarContent,nutrition.sugarContent.g
0,25.2 g,25.2
1,8.9 g,8.9
2,15.5 g,15.5


In [7]:
# check
data[['nutrition.calories', 'nutrition.calories.kcal']].head(3)

,nutrition.calories,nutrition.calories.kcal
0,108.1 calories,108.1
1,195.6 calories,195.6
2,259 calories,259.0


In [8]:
# select only the relevant columns
data = data.iloc[:, [0, 1, 2, 4, 5, 36, 35]]

In [9]:
# inspect
data.shape

(3391, 7)

In [10]:
# drop rows that have any null values
data.dropna(axis=0, how='any', inplace=True)

In [11]:
# inspect
data.shape

(3355, 7)

In [12]:
data.columns.tolist()

['recipe_id',
 'title',
 'date_published',
 'avg_rating',
 'ratings_no',
 'nutrition.calories.kcal',
 'nutrition.sugarContent.g']

# Part II: interactive Dash

NB Need to restart the kernel before running the below script

### Interactivity with callbacks<br>(a tweaked version of the script)

Interactive filters:
- Rating metric
- Nutrition parameter

In [13]:
# data is carried over from before

In [14]:
import dash
from dash import dcc
from dash import html
from jupyter_dash import JupyterDash 
import pandas as pd
import plotly.express as px

import numpy as np
from dash.dependencies import Output, Input

In [15]:
external_stylesheets = [
    {
        "href": "https://fonts.googleapis.com/css2?"
        "family=Lato:wght@400;700&display=swap",
        "rel": "stylesheet",
    },
]

In [16]:
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)
app.title = "Recipes: is it sugar or calories?"

In [17]:
app.layout = html.Div(
    children=[
        
        # header
        html.Div(
            children=[
                html.Img(
                    src=r'assets/apple_pie_logo.jpg', alt='image', className="header-emoji", height="125"),
                html.H1(
                    children="Recipe explorations", className="header-title"),
                html.P(
                    children=["Analyse the behavior of recipe ratings on allrecipes.com", html.Br(), "based on their nutritional values"],
                    className="header-description")],
            className="header"),

        # inputs (dropdown menus)
        html.Div(
            children=[
                             
                
                # dropdown menu to choose ratings parameter
                html.Div(
                    children=[
                        html.Div(children="Rating parameter", className="menu-title"),
                        
                        dcc.Dropdown(
                            options=[
                                {"label": nice_name, "value": col_name}
                                for (nice_name, col_name) in [('Average rating', 'avg_rating'), ('Number of ratings', 'ratings_no')]],
                            value = 'avg_rating', # default value
                            id='rating-filter',
                            clearable=False,
                            className="dropdown")]),
                
                
                # dropdown menu to choose nutrition parameter
                html.Div(
                    children=[
                        html.Div(children="Nutrition parameter", className="menu-title"),
                        
                        dcc.Dropdown(
                            options=[
                                {"label": nice_name, "value": col_name}
                                for (nice_name, col_name) in [('Calories', 'nutrition.calories.kcal'), ('Sugar content', 'nutrition.sugarContent.g')]],
                            value = 'nutrition.calories.kcal', # default value
                            id='nutrient-filter',
                            clearable=False,
                            className="dropdown")]),
            ],
            className="menu",
        ),
        
        # graph
        html.Div(
            children=[
                html.Div(
                    children=dcc.Graph(
                        id="recipe-chart", config={"displayModeBar": False},
                    ),
                    className="card",
                ),
            ],
            className="wrapper",
        ),
    ]
)

In [18]:
@app.callback(
    Output("recipe-chart", "figure"),
    [
        Input("nutrient-filter", "value"),
        Input("rating-filter", "value")
    ],
)


def update_charts(nutrient_choice, rating_choice):
    
    # data range
    filtered_data = data[[nutrient_choice, rating_choice]]

    
    # chart text (chart title, axes titles, hover over text)    
    if nutrient_choice == 'nutrition.calories.kcal':
        x_axis_title = 'Calories per serving (kcal)'
        x_hover = 'calories'
        hov_text_x = '%{x:.0f} calories<extra></extra>'
    elif nutrient_choice == 'nutrition.sugarContent.g':
        x_axis_title = 'Sugar content per serving (grams)'
        hov_text_x = '%{x:.0f} g sugar<extra></extra>'
    
    
    if rating_choice == 'avg_rating':
        chart_title = 'How many stars?'
        y_axis_title = 'Average rating'
        hov_text_y = '%{y:.1f} stars for '
    elif rating_choice == 'ratings_no':
        chart_title = 'How many ratings?'
        y_axis_title = 'Number of ratings'
        hov_text_y = '%{y:.0f} total ratings for '
        
       # '%{y:.1f} %{customdata} for %{x:.1f} %{customdata[1]}<extra></extra>'

    # update figure
    fig = px.scatter(filtered_data,
                     x=nutrient_choice,
                     y=rating_choice)

    fig.update_traces(marker=dict(symbol='circle',
                                  color='rgba(235, 195, 35, 0.25)',
                                  line=dict(width=0.25)),
                      hovertemplate = hov_text_y + hov_text_x)

    fig.update_xaxes(showgrid=True, gridwidth=0.05, gridcolor='#E1E5EA',
                     zeroline=True, zerolinewidth=1, zerolinecolor='Black')
    fig.update_yaxes(showgrid=True, gridwidth=0.05, gridcolor='#E1E5EA',
                     zeroline=True, zerolinewidth=1, zerolinecolor='Black')

    fig.update_layout(title=dict(text=chart_title,
                                            x=0.05,
                                            xanchor='left'),
                                 xaxis=dict(fixedrange=True,
                                            title=x_axis_title),
                                 yaxis=dict(fixedrange=True,
                                            title=y_axis_title),
                                 #paper_bgcolor='rgba(0,0,0,0)',
                                 plot_bgcolor='rgba(0,0,0,0)')

    
    return fig

In [19]:
if __name__ == "__main__":
    app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/
